In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qec import RotatedSurfaceCode

In [2]:
rsc = RotatedSurfaceCode(
    distance = 3,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

In [3]:
rsc.build_memory_circuit(number_of_rounds=1)
rsc.memory_circuit.diagram()

q0: -R-DEPOLARIZE1(0.01)-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.01)-----------------------------------------------------------------------------------------------------------------------------------------------------------------@-DEPOLARIZE2(0.01)--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------DEPOLARIZE1(0.01)--M:rec[8]--------------------------------------------------------------------------------------------------------------------------------------------------------------------------DETECTOR:D10=rec[8]*rec[9]*rec[11]*rec[12]*rec[1]-DETECTOR:D11=rec[8]*rec[9]*rec[11]*rec[12]*rec[1]*rec[1]------------------------------------------------------------DETECTOR:D16=rec[8]*rec[11]*rec[4]------------------OBSERVABLE_INCLUDE:L0*=rec[8]--
                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | |                                                                                                                                                                                 | |
 q1: ---R-----------------DEPOLARIZE1(0.01)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------@-----------------DEPOLARIZE2(0.01)-----------------------------------------|-|---------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.01)-----------------------------------------|-|---------------------------------------------------------X-DEPOLARIZE2(0.01)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------DEPOLARIZE1(0.01)-M:rec[9]----------------------------------------------------------------------------------------------------------------------DETECTOR:D8=rec[9]*rec[10]*rec[0]-DETECTOR:D9=rec[9]*rec[10]*rec[0]*rec[0]------------------------------------------------------------------------------------------------------------------------------DETECTOR:D17=rec[9]*rec[12]*rec[10]*rec[13]*rec[5]--OBSERVABLE_INCLUDE:L0*=rec[9]--
                                                                                                                                                                         

In [5]:
num_shots = 1
num_errors_sampled = rsc.compute_logical_errors(num_shots)

ValueError: The circuit contains non-deterministic detectors.

To make an SVG picture of the problem, you can use the python API like this:
    your_circuit.diagram('detslice-with-ops-svg', tick=range(0, 5), filter_coords=['D12', 'D13', ])
or the command line API like this:
    stim diagram --in your_circuit_file.stim --type detslice-with-ops-svg --tick 0:5 --filter_coords D12:D13 > output_image.svg

This was discovered while analyzing a Z-basis reset (R) on:
    qubit 12

The collapse anti-commuted with these detectors/observables:
    D12
    D13

The backward-propagating error sensitivity for D12 was:
    Y4
    Y5
    Y7
    Y8
    Z10
    X11
    X12

The backward-propagating error sensitivity for D13 was:
    Z4
    Z5
    Z7
    Z8
    Z10
    X12

Circuit stack trace:
    at instruction #31 [which is R 12]

In [4]:
num_shots = 10**2
prob_errors = np.linspace(0, 0.01, 5)

collected_stats = {}

for distance in [3, 5]:
    
    temp_logical_error_rate = []
    
    for prob_error in prob_errors:
        
        rsc = RotatedSurfaceCode(
            distance = distance,
            depolarize1_rate = prob_error,
            depolarize2_rate = prob_error
        )
        rsc.build_memory_circuit(number_of_rounds=distance*3)
        num_errors_sampled = rsc.compute_logical_errors(num_shots)
        temp_logical_error_rate.append(num_errors_sampled / num_shots)
        
    collected_stats[distance] = temp_logical_error_rate

ValueError: The circuit contains non-deterministic detectors.

To make an SVG picture of the problem, you can use the python API like this:
    your_circuit.diagram('detslice-with-ops-svg', tick=range(0, 5), filter_coords=['D76', 'D77', ])
or the command line API like this:
    stim diagram --in your_circuit_file.stim --type detslice-with-ops-svg --tick 0:5 --filter_coords D76:D77 > output_image.svg

This was discovered while analyzing a Z-basis reset (R) on:
    qubit 12

The collapse anti-commuted with these detectors/observables:
    D76
    D77

The backward-propagating error sensitivity for D76 was:
    Y4
    Y5
    Y7
    Y8
    Z10
    X11
    X12

The backward-propagating error sensitivity for D77 was:
    Z4
    Z5
    Z7
    Z8
    Z10
    X12

Circuit stack trace:
    at instruction #855 [which is R 12]

In [ ]:
fig, ax = plt.subplots(1, 1)

for distance in collected_stats.keys():
    ax.plot(prob_errors, collected_stats[distance], label=f"d={distance}")
    
ax.set_ylim(5e-2, 5e-1)
ax.set_xlim(5e-2, 1e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120)